In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
if str(os.getcwd()).endswith('BertModel'):
    os.chdir("..")
import sentencepiece
from BertModel.Sampling import DataSampling
from BertModel.Analyzer import BertAnalyzer
from BertModel.PreTrainedBert import model

/vol/bitbucket/hx224/70016_nlp_coursework/nlpvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = 'dontpatronizeme_pcl.tsv'
titles = ['par_id', 'art_id', 'keyword','country_code','text','label']
raw_data = pd.read_csv(path, skiprows = 4, sep = '\t',
                       names = titles)
raw_data = raw_data.dropna()
raw_data['label'] = np.where(raw_data['label'] > 1, 1, 0)

In [3]:
# Train - validation (dev) - test split
train = pd.read_csv("semeval-2022/practice splits/train_semeval_parids-labels.csv")
test = pd.read_csv("semeval-2022/practice splits/dev_semeval_parids-labels.csv")
train_df_official = raw_data[raw_data["par_id"].isin(train['par_id'])]
test_df = raw_data[raw_data["par_id"].isin(test['par_id'])]

train_data_shuffled = train_df_official.sample(frac = 1, random_state = 1).reset_index(drop = True)
split_index = int(0.8 * len(train_data_shuffled))

train_df = train_data_shuffled.iloc[:split_index]
val_df = train_data_shuffled.iloc[split_index:]

In [4]:
datasampling = DataSampling()
upsample_2 = datasampling.upsampling_with_mask_and_fill(raw_data=train_df, mask_ratio=0.2)
upsample_4 = datasampling.upsampling_with_mask_and_fill(raw_data=train_df, mask_ratio=0.4)
upsample_6 = datasampling.upsampling_with_mask_and_fill(raw_data=train_df, mask_ratio=0.6)
upsample_8 = datasampling.upsampling_with_mask_and_fill(raw_data=train_df, mask_ratio=0.8)
upsample_2.to_csv("upsample_2.csv")
upsample_4.to_csv("upsample_4.csv")
upsample_6.to_csv("upsample_6.csv")
upsample_8.to_csv("upsample_8.csv")

Device set to use cuda:0
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Processing Keywords: 100%|██████████| 10/10 [04:08<00:00, 24.86s/it]


In [6]:
save_dir = "xlnet_analyzer_train_save"
mask_ratios = [0.2, 0.4, 0.6, 0.8]
data_files = ["upsample_2.csv", "upsample_4.csv", "upsample_6.csv", "upsample_8.csv"]

# Track best model
best_f1 = 0
best_model_path = None
best_mask_ratio = None

for mask_ratio, file_path in zip(mask_ratios, data_files):
    save_path = os.path.join(save_dir, f"xlnet_analyzer_model_mask_{int(mask_ratio * 10)}.pth")

    # Load the dataset from CSV
    train_df = pd.read_csv(file_path)  # FIX: Load CSV before passing to BertAnalyzer.train()

    # Initialize model
    xlnet_model = model("xlnet-base-cased")

    xlnet_analyzer = BertAnalyzer(model=xlnet_model,
                                  batch_size=64,
                                  max_seq_len=128,
                                  epochs=5,
                                  lr=4e-05)

    print(f"Training with mask_ratio={mask_ratio}, saving model to {save_path}")

    # Train and save the model
    xlnet_analyzer.train(data_file=train_df, save_path=save_path)  # Pass the loaded DataFrame

    # Evaluate model on validation set (val_df)
    f1_score = xlnet_analyzer.evaluate(val_df)  # val_df should NOT be upsampled

    print(f"F1 Score on val_df for mask_ratio={mask_ratio}: {f1_score:.4f}")
    print("-" * 50)

    # Track the best model
    if f1_score > best_f1:
        best_f1 = f1_score
        best_model_path = save_path
        best_mask_ratio = mask_ratio
        
    del xlnet_model
    del xlnet_analyzer
    torch.cuda.empty_cache()

# Print the best model and mask ratio
print(f"Best Model: {best_model_path} with mask_ratio={best_mask_ratio} and F1 Score={best_f1:.4f}")


Training with mask_ratio=0.2, saving model to xlnet_analyzer_train_save/xlnet_analyzer_model_mask_2.pth


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

epoch:, 1 -- batch: 10 -- avg loss: 0.6503
epoch:, 1 -- batch: 20 -- avg loss: 0.3246
epoch:, 1 -- batch: 30 -- avg loss: 0.2330
epoch:, 1 -- batch: 40 -- avg loss: 0.1929
epoch:, 1 -- batch: 50 -- avg loss: 0.1670
epoch:, 1 -- batch: 60 -- avg loss: 0.1869
epoch:, 1 -- batch: 70 -- avg loss: 0.1232
epoch:, 1 -- batch: 80 -- avg loss: 0.1537
epoch:, 1 -- batch: 90 -- avg loss: 0.1536
epoch:, 1 -- batch: 100 -- avg loss: 0.1383
epoch:, 1 -- batch: 110 -- avg loss: 0.1131
epoch:, 1 -- batch: 120 -- avg loss: 0.1419
epoch:, 1 -- batch: 130 -- avg loss: 0.1306
epoch:, 1 -- batch: 140 -- avg loss: 0.0915
epoch:, 1 -- batch: 150 -- avg loss: 0.1214
epoch:, 1 -- batch: 160 -- avg loss: 0.1210
epoch:, 1 -- batch: 170 -- avg loss: 0.1244
epoch:, 1 -- batch: 180 -- avg loss: 0.1396
epoch:, 1 -- batch: 190 -- avg loss: 0.1046


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

epoch:, 2 -- batch: 10 -- avg loss: 0.0586
epoch:, 2 -- batch: 20 -- avg loss: 0.0419
epoch:, 2 -- batch: 30 -- avg loss: 0.0321
epoch:, 2 -- batch: 40 -- avg loss: 0.0315
epoch:, 2 -- batch: 50 -- avg loss: 0.0356
epoch:, 2 -- batch: 60 -- avg loss: 0.0609
epoch:, 2 -- batch: 70 -- avg loss: 0.0346
epoch:, 2 -- batch: 80 -- avg loss: 0.0427
epoch:, 2 -- batch: 90 -- avg loss: 0.0415
epoch:, 2 -- batch: 100 -- avg loss: 0.0517
epoch:, 2 -- batch: 110 -- avg loss: 0.0468
epoch:, 2 -- batch: 120 -- avg loss: 0.0568
epoch:, 2 -- batch: 130 -- avg loss: 0.0419
epoch:, 2 -- batch: 140 -- avg loss: 0.0474
epoch:, 2 -- batch: 150 -- avg loss: 0.0300
epoch:, 2 -- batch: 160 -- avg loss: 0.0376
epoch:, 2 -- batch: 170 -- avg loss: 0.0632
epoch:, 2 -- batch: 180 -- avg loss: 0.0513
epoch:, 2 -- batch: 190 -- avg loss: 0.0239


...

epoch:, 4 -- batch: 10 -- avg loss: 0.0147
epoch:, 4 -- batch: 20 -- avg loss: 0.0059
epoch:, 4 -- batch: 30 -- avg loss: 0.0092
epoch:, 4 -- batch: 40 -- avg loss: 0.0063
epoch:, 4 -- batch: 50 -- avg loss: 0.0044
epoch:, 4 -- batch: 60 -- avg loss: 0.0072
epoch:, 4 -- batch: 70 -- avg loss: 0.0028
epoch:, 4 -- batch: 80 -- avg loss: 0.0040
epoch:, 4 -- batch: 90 -- avg loss: 0.0014
epoch:, 4 -- batch: 100 -- avg loss: 0.0024
epoch:, 4 -- batch: 110 -- avg loss: 0.0046
epoch:, 4 -- batch: 120 -- avg loss: 0.0028
epoch:, 4 -- batch: 130 -- avg loss: 0.0038
epoch:, 4 -- batch: 140 -- avg loss: 0.0166
epoch:, 4 -- batch: 150 -- avg loss: 0.0348
epoch:, 4 -- batch: 160 -- avg loss: 0.0291
epoch:, 4 -- batch: 170 -- avg loss: 0.0179
epoch:, 4 -- batch: 180 -- avg loss: 0.0117
epoch:, 4 -- batch: 190 -- avg loss: 0.0092


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

epoch:, 5 -- batch: 10 -- avg loss: 0.0169
epoch:, 5 -- batch: 20 -- avg loss: 0.0079
epoch:, 5 -- batch: 30 -- avg loss: 0.0040
epoch:, 5 -- batch: 40 -- avg loss: 0.0161
epoch:, 5 -- batch: 50 -- avg loss: 0.0055
epoch:, 5 -- batch: 60 -- avg loss: 0.0089
epoch:, 5 -- batch: 70 -- avg loss: 0.0067
epoch:, 5 -- batch: 80 -- avg loss: 0.0059
epoch:, 5 -- batch: 90 -- avg loss: 0.0078
epoch:, 5 -- batch: 100 -- avg loss: 0.0088
epoch:, 5 -- batch: 110 -- avg loss: 0.0039
epoch:, 5 -- batch: 120 -- avg loss: 0.0089
epoch:, 5 -- batch: 130 -- avg loss: 0.0064
epoch:, 5 -- batch: 140 -- avg loss: 0.0082
epoch:, 5 -- batch: 150 -- avg loss: 0.0068
epoch:, 5 -- batch: 160 -- avg loss: 0.0032
epoch:, 5 -- batch: 170 -- avg loss: 0.0085
epoch:, 5 -- batch: 180 -- avg loss: 0.0094
epoch:, 5 -- batch: 190 -- avg loss: 0.0046


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Accuracy: 0.9182
Confusion Matrix:
[[1493   32]
 [ 105   45]]
f1
0.3964757709251101
F1 Score on val_df for mask_ratio=0.8: 0.3965
--------------------------------------------------
Best Model: xlnet_analyzer_train_save/xlnet_analyzer_model_mask_2.pth with mask_ratio=0.2 and F1 Score=0.4247
